<a href="https://colab.research.google.com/github/Baronvera/MetagenomicaColab/blob/main/Copia_de_Metagenomica_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**2. Instalación de Conda y programas**

Instale y llame a conda.

In [ ]:
pip install -q condacolab

/bin/bash: line 1: pip: command not found


In [ ]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!conda --version

conda 24.11.2


Instale sus programas.

In [ ]:
!conda install -c conda-forge -c bioconda -c defaults abricate

Channels:
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - abricate


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    abricate-1.0.1             |       h05cac1d_4        13.2 MB  bioconda
    alsa-lib-1.2.11            |       hd590300_1         542 KB  conda-forge
    any2fasta-0.4.2            |       hdfd78af_3           8 KB  bioconda
    argtable2-2.13             |    hd590300_1004         2.0 MB  conda-forge
    blast-2.16.0               |       hc155240_3       141.1 MB  bioconda
    blast-legacy-2.2.26        |       hf7ff83a_5        44.3 MB  bioconda
    ca-certificates-2025.4.26  |       hbd8a1cb_0         149 KB  conda-forge
    cairo-1.18.0               |       h3faef2a_0         959 KB  conda-forge
    certifi-202

In [ ]:
!abricate --check

Checking dependencies are installed:
Found 'blastn' => /usr/local/bin/blastn
Found 'blastx' => /usr/local/bin/blastx
Found 'makeblastdb' => /usr/local/bin/makeblastdb
Found 'blastdbcmd' => /usr/local/bin/blastdbcmd
Found 'any2fasta' => /usr/local/bin/any2fasta
Found 'gzip' => /usr/bin/gzip
Found 'unzip' => /usr/local/bin/unzip
OK.


In [ ]:
!conda install bioconda::flye

Channels:
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - bioconda::flye


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    flye-2.9.5                 |  py311hc849397_1        34.9 MB  bioconda
    ------------------------------------------------------------
                                           Total:        34.9 MB

The following NEW packages will be INSTALLED:

  flye               bioconda/linux-64::flye-2.9.5-py311hc849397_1 



                                                                        
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - done


In [ ]:
%%shell
conda install -c bioconda quast -y

Channels:
 - bioconda
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - quast


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bedtools-2.31.1            |       hf5e1c6e_2         1.5 MB  bioconda
    brotli-1.1.0               |       hb9d3cd8_2          19 KB  conda-forge
    brotli-bin-1.1.0           |       hb9d3cd8_2          18 KB  conda-forge
    bwa-0.7.18                 |       he4a0461_1         192 KB  bioconda
    circos-0.69.9              |       hdfd78af_0        26.3 MB  bioconda
    contourpy-1.3.2            |  py311hd18a35c_0         272 KB  conda-forge
    cycler-0.12.1              |     pyhd8ed1ab_1          13 KB  conda-forge
    fonttools-4.58.1           |  py311h2dc5d0c_0         2.8 MB  conda-forge
    gli

In [ ]:
!conda create -yqn tutorial -c conda-forge -c bioconda kraken2 krakentools bracken
conda_path = !source activate tutorial && echo $CONDA_PREFIX:$PATH
%set_env PATH=$conda_path

Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local/envs/tutorial

  added / updated specs:
    - bracken
    - kraken2
    - krakentools


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    biopython-1.85             |  py313h536fd9c_1         3.4 MB  conda-forge
    blast-2.16.0               |       h66d330f_5       141.4 MB  bioconda
    bracken-3.1                |       h9948957_0         495 KB  bioconda
    c-ares-1.34.5              |       hb9d3cd8_0         202 KB  conda-forge
    curl-8.14.0                |       h332b0f4_0         178 KB  conda-forge
    kraken2-2.14               | pl5321h077b44d_0         3.3 MB  bioconda
    krakentools-1.2            |     pyh7e72e81_1          36 KB  bioconda
    ld_impl_linux-64-2.43      |       h712a8e2_4         656 KB  conda-forge
    

In [ ]:
!conda install -c bioconda krona -y

Channels:
 - bioconda
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - krona


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    krona-2.8.1                | pl5321hdfd78af_1         155 KB  bioconda
    ------------------------------------------------------------
                                           Total:         155 KB

The following NEW packages will be INSTALLED:

  krona              bioconda/noarch::krona-2.8.1-pl5321hdfd78af_1 



                                                                        
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ | / - \ | / - \ | / - \ | / 
Krona installed.  You still need to manually update the taxonomy
databa

#**3. Ensamblaje (metaFlye)**

A continuación usted realizará el ensamblaje *de novo* del metagenoma de la muestra procesada. En este paso se utilizará el programa `flye` con habilitando la opción para metagenomas `--meta`.

**Nota:**
- Esto tarda aproximadamente 40 minutos para el archivo completo `MetaG.fastq`).
- El archivo `MetaG_50p.fastq` es subset de datos que corresponde al 50% de las lecturas y tarda solo 15 minutos.
- El archivo `MetaG_30p.fastq` es subset de datos que corresponde al 30% de las lecturas y tarda solo 5 minutos.

Descargue los archivos fastq.

In [ ]:
!wget -O MetaG_30p.fastq https://www.dropbox.com/scl/fi/z7ia4jbkpzb4pvwwiuz2v/MetaG_30p.fastq?rlkey=0eyuhu0lz7lp4hb2nt1xnab2y&st=znlx6st4&dl=0

--2025-05-30 15:25:49--  https://www.dropbox.com/scl/fi/z7ia4jbkpzb4pvwwiuz2v/MetaG_30p.fastq?rlkey=0eyuhu0lz7lp4hb2nt1xnab2y
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7ac176a8f3c02dd1dfa7b864d3.dl.dropboxusercontent.com/cd/0/inline/CqqG3goSWjxUoH-9DdW1WzXf5PFBk9kVjkBm2n8C4uX3jNY75hskysMi-3TmcM3H5ryMfK0o6rgKnnbvI0PRnjUJhbAhr5mTv6p7VxM0kSaelje5-Y5pF-fbGcnxzuUCD_bkBI9-6IyIPEAl09yQvKLx/file# [following]
--2025-05-30 15:25:50--  https://uc7ac176a8f3c02dd1dfa7b864d3.dl.dropboxusercontent.com/cd/0/inline/CqqG3goSWjxUoH-9DdW1WzXf5PFBk9kVjkBm2n8C4uX3jNY75hskysMi-3TmcM3H5ryMfK0o6rgKnnbvI0PRnjUJhbAhr5mTv6p7VxM0kSaelje5-Y5pF-fbGcnxzuUCD_bkBI9-6IyIPEAl09yQvKLx/file
Resolving uc7ac176a8f3c02dd1dfa7b864d3.dl.dropboxusercontent.com (uc7ac176a8f3c02dd1dfa7b864d3.dl.dropboxusercontent.com)... 162.125.3.15, 262

In [ ]:
!wget -O MetaG_50p.fastq https://www.dropbox.com/scl/fi/pedbg1mrdgv1e7tlkhkfh/MetaG_50p.fastq?rlkey=m7m899j6rjtufb28jikpq620k&st=uv6vy5a3&dl=0

--2025-05-30 15:26:46--  https://www.dropbox.com/scl/fi/pedbg1mrdgv1e7tlkhkfh/MetaG_50p.fastq?rlkey=m7m899j6rjtufb28jikpq620k
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf4a0faa77caa712bf7038d0ce3.dl.dropboxusercontent.com/cd/0/inline/Cqq_bl6KX6KTKxtoDAEow8ZvakIAQLtPdYDFaJDZc7kysUVBjzwhJxxdgce7_G9SrJ7vJOS7hTu7rc0ef4dghA94taNyoYYo-nE-JCeJL69chCpovM56Gy6UOuuQckcyxXbwPmtzfOmLei9o4R16SvYW/file# [following]
--2025-05-30 15:26:47--  https://ucf4a0faa77caa712bf7038d0ce3.dl.dropboxusercontent.com/cd/0/inline/Cqq_bl6KX6KTKxtoDAEow8ZvakIAQLtPdYDFaJDZc7kysUVBjzwhJxxdgce7_G9SrJ7vJOS7hTu7rc0ef4dghA94taNyoYYo-nE-JCeJL69chCpovM56Gy6UOuuQckcyxXbwPmtzfOmLei9o4R16SvYW/file
Resolving ucf4a0faa77caa712bf7038d0ce3.dl.dropboxusercontent.com (ucf4a0faa77caa712bf7038d0ce3.dl.dropboxusercontent.com)... 162.125.3.15, 262

Observe las primeras lineas de su archivo.

In [ ]:
!head MetaG_50p.fastq

@b8f3813f-8cba-440a-90ee-957a1c5fb2f7 runid=54890cda89c7a7814fb75e0d761a9c261e0ab086 sampleid=no_sample read=27 ch=293 start_time=2024-06-29T04:58:49Z model_version_id=dna_r9.4.1_e8_hac@v3.3 barcode=barcode49
GTATTACTTCGTTCGGTTACGTATTGCTAAGGTTAAACTGGTGCAGCTTTGAACATCTAGCAGCACCTCACACCGATTTTATATTAAATTGTGGGCCGAATAATCGTTGGATGAGAACATAATGTCCTCTCCTTTACACCCTTAGCATTGCTCAGCATCAGCTCTGTCGAGAGAAACGATCACATCCGCAACCATATTCTCTCAAATATTGATCAGGAACGGGTTGGAGTCGCCTTCACTAACTCCTTGTACCTATTCCATGCAAATGCCTCTTTCAGTGCTTTTGCATTCTCACAGTTCCCTTTTCATCATAGATATGAAACGGTGTGGGATATTTTCTGCAATCTTTTCGATCATTTGCTTTGGTAACAAATGCCTTTTTCCATAAGTATCTCCTCTGAATATATTTATTACAATAAAGTACTTCTCATCTCTTCCAGGTCCGTGCCAAATGTCAGGAGCAACATCAATGATCTGCGCCCTTCTGACCTTCCTGGTTCATACGATAGGCTGCTCTTGCAACAGTCGTACACGGGAAGTAAATTCGAGGTGCTGCTAGATGTTTCAAAGCTGCACCAGTTTAACCTTAGCAATACGTAATA
+
#$&)*()++/..89''().//.../0020*(*,-025673332265667766799=>>?:::665799910*,36:==:2'%%''$$%%%$$%',,0.(((''%,0674>>=5568866677:<?=323347+3333578+()),56;91000*(())/1-.1152)&%%%******('()))(+:

Ejecute flye.

In [ ]:
%%shell
flye --meta --nano-raw MetaG_50p.fastq --out-dir /content/assembly --threads 2

[2025-05-30 15:27:01] INFO: Starting Flye 2.9.5-b1801
[2025-05-30 15:27:01] INFO: >>>STAGE: configure
[2025-05-30 15:27:01] INFO: Configuring run
[2025-05-30 15:27:02] INFO: Total read length: 83368087
[2025-05-30 15:27:02] INFO: Reads N50/N90: 8556 / 485
[2025-05-30 15:27:02] INFO: Minimum overlap set to 1000
[2025-05-30 15:27:02] INFO: >>>STAGE: assembly
[2025-05-30 15:27:02] INFO: Assembling disjointigs
[2025-05-30 15:27:02] INFO: Reading sequences
[2025-05-30 15:27:12] INFO: Counting k-mers:
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2025-05-30 15:28:45] INFO: Filling index table (1/2)
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2025-05-30 15:29:15] INFO: Filling index table (2/2)
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2025-05-30 15:29:48] INFO: Extending reads
[2025-05-30 15:29:52] INFO: Overlap-based coverage: 1
[2025-05-30 15:29:52] INFO: Median overlap divergence: 0.0862551
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2025-05-30 15:30:27] INFO: Assembled 780 disjoi

#**4. Evaluación de los genomas ensamblados.**

##**4.1. Clasificación taxonómica**

Primero que todo deberemos identificar taxonómicamente los genomas obtenidos. Podemos encontrar varias opciones, entre estas podemos hacer uso del programa `kraken2`.

###**4.1.2. Descargar base de datos**

Primero deberemos descargar la base de datos de refencia. La decisión de cual base de datos usar influenciará en la cantidad de contigs identificados, asi que sea cauteloso en esta decisión.

**Nota:** Este proceso tarda aproximadamente 8 minutos.

In [ ]:
!mkdir protocol_db
!wget -qnc https://genome-idx.s3.amazonaws.com/kraken/k2_standard_eupath_20201202.tar.gz
!tar xvfz k2_standard_eupath_20201202.tar.gz -C protocol_db

hash.k2d
opts.k2d
taxo.k2d
seqid2taxid.map
inspect.txt
database100mers.kmer_distrib
database150mers.kmer_distrib
database200mers.kmer_distrib
database250mers.kmer_distrib
database300mers.kmer_distrib
database50mers.kmer_distrib
database75mers.kmer_distrib


###**4.1.3. Clasifique los contigs con Kraken2**

A continuación realice la identificación taxonómica de los contigs, obtendrá un archivo de salida y un reporte.

In [ ]:
!kraken2 --db protocol_db --report metaG.k2report --minimum-hit-groups 3 /content/assembly/assembly.fasta > metaG.kraken2

Loading database information... done.
66 sequences (1.38 Mbp) processed in 0.104s (38.0 Kseq/m, 791.84 Mbp/m).
  58 sequences classified (87.88%)
  8 sequences unclassified (12.12%)


Observe el número de contigs clasificados.

###**4.1.4. Estime las abundancias con Bracken**

In [ ]:
!bracken -d protocol_db -i metaG.k2report -o metaG.bracken -w metaG.breport -r 100 -l S -t 2

/bin/bash: line 1: bracken: command not found


Observe el resultado de bracken.

In [ ]:
!head metaG.bracken

name	taxonomy_id	taxonomy_lvl	kraken_assigned_reads	added_reads	new_est_reads	fraction_total_reads
Lachnospira eligens	39485	S	7	0	7	0.19444
Blautia sp. SC05B48	2479767	S	2	0	2	0.05556
Faecalibacterium prausnitzii	853	S	5	1	6	0.16667
Bacteroides uniformis	820	S	4	0	4	0.11111
Bacteroides fragilis	817	S	3	0	3	0.08333
Bacteroides thetaiotaomicron	818	S	3	0	3	0.08333
Bacteroides heparinolyticus	28113	S	2	0	2	0.05556
Phocaeicola dorei	357276	S	2	0	2	0.05556
Escherichia coli	562	S	4	1	5	0.13889


###**4.1.5. Visualice la taxonomía con Krona**

Cambie el formato de bracken a krona.

In [ ]:
!kreport2krona.py -r metaG.breport -o metaG.b.krona.txt --no-intermediate-ranks

Actualice la base de datos de taxonomía de krona.

In [ ]:
!ktUpdateTaxonomy.sh

Fetching taxdump.tar.gz...
   Fetching checksum...
   Checksum for taxdump.tar.gz matches server.
Extracting taxonomy...

Cleaning up...

Finished.



Genere el reporte de krona.

In [ ]:
!ktImportTaxonomy -t 2 -m 6 metaG.bracken -o metaG.krona.out.html

Loading taxonomy...
Importing metaG.bracken...
Writing metaG.krona.out.html...


Descargue el reporte y visualice el resultado directamente en algún navegador.

In [ ]:
from google.colab import files
files.download('metaG.krona.out.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download('metaG.breport')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###**4.1.6. Visualice los resultados con Pavian**

Usted también podrá usar los archivos de reporte de bracken para explorar los resultados en la página web de Pavian.

In [ ]:
from IPython.display import IFrame
IFrame('https://fbreitwieser.shinyapps.io/pavian/', width='100%', height=600)

##**5.2. Estadísticas del ensamblaje**

Primero podemos descargar la base de datos de silva para quast. Nos será de utilidad más adelante.

In [ ]:
%%bash
quast-download-silva

Couldn't find program: 'bash'


Observe las opciones de `quast` desplegando el menú de help.

In [ ]:
!metaquast.py -h

Corra `metaquast` especificando los genomas de refencia que desea analizar desde el archivo `list_references.txt` que se encuentra en la carpeta `data` dentro de `Modulo_4`. Usted puede crear este archivo usando el resultado obtenido en krake2/bracken.

**Nota:** Esto tarda aproximadamente 12 minutos para el 100%
y 4 minutos para 50%.

Descargue el archivo con la lista de especies.

In [ ]:
!wget -O list_references_50p.txt https://www.dropbox.com/scl/fi/m8ow3goeyfvx6it16pln4/list_references_50p.txt?rlkey=fv93p4jfgjzfovmmtab9r62oi&st=j4c9hbl7&dl=0

Ejecute metaquast.

In [ ]:
%%shell
metaquast.py /content/assembly/assembly.fasta -t 2 --references-list list_references_50p.txt

Otra opción es ejecutar este comando pero en lugar de especificar una lista de genomas usted puede usar la opción `--gene-finding`, el cual contrastará los contigs contra la base de datos de silva que previamente descargamos para identificar los genomas de referencia a descargar.

**Nota:** Esto tarda aproximadamente 15 minutos.

**Nota2:** No haremos esta opción por eso esta como un comentario (#), pero si desea puede hacerlo y comparar los resultados de ambas opciones.

In [ ]:
%%shell
#metaquast.py /content/assembly/assembly.fasta -t 2 --gene-finding

In [ ]:
%ls /content/quast_results/latest/

In [ ]:
import IPython
IPython.display.HTML(filename='/content/quast_results/latest/report.html')

In [ ]:
import IPython
IPython.display.HTML(filename='/content/quast_results/latest/icarus.html')

# **5. ABRicate. Resistencia a antibióticos**

Identifica genes de resistencia a antibióticos mediante la ejecución de ensamblajes locales.

Cribado masivo de contigs para genes de resistencia a los antimicrobianos o de virulencia. Incluye varias bases de datos: `NCBI`, `CARD`, `ARG-ANNOT`, `Resfinder`, `MEGARES`, `EcOH`, `PlasmidFinder`, `Ecoli_VF` y `VFDB`.

¿Es esta la herramienta adecuada para mí?
- Solo admite contigs, no lecturas FASTQ
- Solo detecta genes de resistencia adquiridos, **NO** mutaciones puntuales
- Utiliza una base de datos de secuencias de ADN, no de proteínas.
- Necesita BLAST+ >= 2.7 y any2fastaestar instalado
- Está escrito en Perl 🐫

Antes de ejecutar el programa confirme si funciona correctamente y que cumple con todos los requisitos.

In [ ]:
!abricate --check

El error puede ser ocasionado por el cambio del enviroment cuando instalamos `kraken2`. Vuelva a colocarlo en base.

In [ ]:
!echo $PATH

In [ ]:
%set_env PATH=['/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin' ]

In [ ]:
!echo $PATH

Observe las bases de datos disponibles para ABRicate.

In [ ]:
!abricate --list

Usted puede actualizar cada una de las bases de datos disponibles con el comando `abricate-get_db`.

In [ ]:
!abricate-get_db --help

In [ ]:
!abricate-get_db --db card --force

Observe la actualización de los registros de la base de datos.

In [ ]:
!abricate --list

Abricate acepta cualquier archivo de secuencia que any2fasta pueda convertirse a archivos FASTA (por ejemplo, Genbank, EMBL) y pueden ser opcionalmente gzipcomprimidos bzip2.

**Nota:** No acepta lecturas FASTQ sin procesar; utilice otros programas como Ariba o SRTS2 para eso.



In [ ]:
!abricate /content/assembly/assembly.fasta --db ncbi > abricate_output_ncbi.tsv

Los resultados son almacenados en una archivo `.tsv`.

In [ ]:
!cat abricate_output_ncbi.tsv

Podemos probar otras bases de datos. Sientase libre de probarlas.

In [ ]:
!abricate /content/assembly/assembly.fasta --db megares > abricate_output_megares.tsv

In [ ]:
!cat abricate_output_megares.tsv

**Advertencias**

- No encuentra resistencia mutacional, sólo genes adquiridos.
- Informe de brechas incompleto
- A veces se informarán dos genes muy superpuestos para el mismo locus.
- Posibles problemas en el cálculo de la cobertura.

La base de datos predeterminada es ncbi. Puede elegir una base de datos diferente mediante la opción `--db`.

Realice el análisis cambiado la base de datos a vcfdb.


In [ ]:
!abricate --db vfdb /content/assembly/assembly.fasta > abricate_output_vfdb.tsv

Observe el resultado con vfdb.

In [ ]:
!cat abricate_output_vfdb.tsv

Ahora con Card.

In [ ]:
!abricate --db card /content/assembly/assembly.fasta > abricate_output_card.tsv

Observe el resultado con Card.

In [ ]:
!cat abricate_output_card.tsv

Si usted desea comparar varias muestras use el comando `summary` y enliste todos los output de las distintas muestras. Para ejemplificar creemos una copia de nuestro resultado, o realice los pasos anteriores para otra muestra.



Creemos la copia.

In [ ]:
!cp abricate_output_ncbi.tsv abricate_output2_ncbi.tsv

Comparemos los resultados.

In [ ]:
!abricate --summary abricate_output_ncbi.tsv abricate_output2_ncbi.tsv > tabla_resultado_ncbi.tsv

In [ ]:
!cat tabla_resultado_ncbi.tsv